In [ ]:
!pip install -q transformers datasets evaluate huggingface_hub peft accelerate bitsandbytes

In [ ]:
!pip install --upgrade --force-reinstall datasets

In [ ]:
from datasets import load_dataset

# Load AG News dataset for a clearer fine-tuning improvement
dataset = load_dataset("ag_news")
dataset = dataset.shuffle(seed=42)
print(dataset)

In [ ]:
!pip list

In [ ]:
print("Sample News:", dataset["train"][0]["text"])
print("Label (0 = World, 1 = Sports, 2 = Business, 3 = Sci/Tech):", dataset["train"][0]["label"])

In [ ]:
from datasets import DatasetDict

# Reduce size for faster demo
small_train = dataset["train"].select(range(4000))
small_test = dataset["test"].select(range(1000))

# Train/Validation Split
small_train_test = small_train.train_test_split(test_size=0.2, seed=42)
datasets = DatasetDict({
    "train": small_train_test["train"],
    "validation": small_train_test["test"],
    "test": small_test
})
print(datasets)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

In [ ]:
print(tokenized_datasets["train"][1])
#1 indicates pay attention to that token and 0 indicates do not pay attention

In [ ]:
from transformers import AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

In [ ]:
target_modules = ["query", "value"]  # key layers in attention blocks
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

In [ ]:
# LoRA rank: defines the dimensionality of the low-rank adapter matrices.
# A higher rank increases model capacity but also adds more trainable parameters.

# LoRA alpha: scaling factor for the LoRA output.
# The actual adapter output is scaled by (alpha / rank) to balance with the frozen base model.

# Target modules: list of specific layer names in the model where LoRA adapters should be injected.
# Typically includes attention-related layers like "q_proj" or "v_proj" in transformer models.

# LoRA dropout: dropout probability applied only to the LoRA adapter output during training.
# Helps reduce overfitting, especially on smaller datasets.

# Bias setting: controls whether and where to train bias terms.
# Options: "none" (do not train any biases), "all" (train all biases), "lora_only" (train only in layers with LoRA).

# Task type: specifies the kind of downstream task for which the model is being fine-tuned.
# Examples: SEQ_CLS (sequence classification), CAUSAL_LM (language modeling), TOKEN_CLS (token classification), etc.

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
print(pipe("The stock market closed higher today after positive earnings reports."))
print(pipe("The player scored a hat trick in the championship game."))
#0 = World, 1 = Sports, 2 = Business, 3 = Sci/Tech

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",                        # Directory to save model checkpoints and outputs
    eval_strategy="epoch",                  # Evaluate at the end of every epoch
    save_strategy="epoch",                        # Save checkpoint at the end of every epoch
    learning_rate=2e-4,                            # Learning rate for optimizer
    per_device_train_batch_size=8,                 # Batch size per device (GPU/TPU) during training
    per_device_eval_batch_size=8,                  # Batch size per device during evaluation
    num_train_epochs=2,                            # Total number of training epochs
    weight_decay=0.01,                             # Weight decay to apply (for regularization)
    logging_dir="./logs",                         # Directory to store logs
    push_to_hub=False,                             # Whether to push model to Hugging Face Hub
    report_to="none"                               # Disable logging to external tools like WandB/Comet
)

In [ ]:
# Load Hugging Face's built-in evaluation library
import evaluate
import numpy as np

# Load the "accuracy" metric — this will download a standard implementation for accuracy scoring
accuracy = evaluate.load("accuracy")

# Define a custom compute_metrics function that the Trainer will use during evaluation
def compute_metrics(eval_pred):
    # eval_pred is a tuple: (model_outputs, true_labels)
    logits, labels = eval_pred

    # Convert logits (raw model outputs) into predicted class indices
    # The model outputs a vector of scores for each class (e.g., [0.2, 0.8])
    # np.argmax selects the index with the highest score, i.e., the predicted label
    predictions = np.argmax(logits, axis=-1)

    # Compare predicted labels with true labels and calculate accuracy
    # accuracy.compute returns a dictionary, e.g., {"accuracy": 0.89}
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
results = trainer.evaluate(tokenized_datasets["test"])
print(results)

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer)
#0 = World, 1 = Sports, 2 = Business, 3 = Sci/Tech
print(pipe("The stock market closed higher today after positive earnings reports."))
print(pipe("The player scored a hat trick in the championship game."))

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

trainer.push_to_hub()